In [13]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

dataset = pd.read_csv ('test2.csv')
dataset2 = pd.read_csv ('test2.csv')

In [14]:
df = dataset[dataset['MAC Source']=='00:0c:29:9d:9e:9e']
df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['Delta Time'].mean()
time_deltas = time_deltas.fillna(time_deltas.mean())
p_per_sec = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['No.'].count()
p_per_sec = time_deltas.fillna(0)
#X = df_s.drop('MAC Source', axis=1)

X = pd.merge(time_deltas, p_per_sec, left_index=True, right_index=True)

model = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination=0.1)
model.fit(X.values)

<ipython-input-14-7c4be75cccc6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')


LocalOutlierFactor(contamination=0.1, novelty=True)

In [15]:
X['scores']=model.decision_function(X[X.columns[0:2]].values)
X['anomaly']=model.predict(X[X.columns[0:2]].values)
anomaly=X.loc[X['anomaly']==-1]
model.fit(anomaly[anomaly.columns[0:2]].values)

C:\Users\carlo\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_lof.py:265: UserWarning: n_neighbors (20) is greater than the total number of samples (5). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "


LocalOutlierFactor(contamination=0.1, novelty=True)

In [16]:
df = dataset2[dataset2['MAC Source']=='00:80:f4:09:51:3b']
df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['Delta Time'].mean()
time_deltas = time_deltas.fillna(time_deltas.mean())
p_per_sec = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['No.'].count()
p_per_sec = time_deltas.fillna(0)

X_test = pd.merge(time_deltas, p_per_sec, left_index=True, right_index=True)
X_final = X_test

X_final['scores'] = model.decision_function(X_test[X_test.columns[0:2]].values)
X_final['anomaly'] = model.predict(X_test[X_test.columns[0:2]].values)
anomaly=X_final.loc[X_final['anomaly']==-1]



<ipython-input-16-581dc75f59ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')


In [17]:
# X_final['scores']=model.decision_function(X_test[X_test.columns[0:2]].values)
# X_final['anomaly']=model.predict(X_test[X_test.columns[0:2]].values)
# anomaly=X_final.loc[X_final['anomaly']==-1]

In [18]:


threshold = (anomaly['scores'].min() + anomaly['scores'].max())/2
anomaly#[anomaly['scores']<threshold]

,Delta Time_x,Delta Time_y,scores,anomaly
Epoch Time,,,,
2018-09-08 20:54:00,0.006298,0.006298,-4.145717,-1
2018-09-08 20:55:00,0.006335,0.006335,-4.143946,-1
2018-09-08 20:56:00,0.005871,0.005871,-4.166022,-1
2018-09-08 20:57:00,0.006600,0.006600,-4.131329,-1
2018-09-08 20:58:00,0.006323,0.006323,-4.144525,-1
...,...,...,...,...
2018-09-08 21:50:00,0.006400,0.006400,-4.140861,-1
2018-09-08 21:51:00,0.005943,0.005943,-4.162586,-1
2018-09-08 21:52:00,0.006298,0.006298,-4.145697,-1
